In [1]:
# %cd ..
import inputs, simuls, analysis
import numpy as np
results = analysis.GetResults(verbose=True)

# PenEasy SPC simulation

In [ ]:
pid = 'SPC'
# Initialize PenEasy Input Editor
peneasy_inps = inputs.PenEasy(verbose=True)

# Initialize Simulators (identified by pid)
peneasy_sims = simuls.PenEasy(verbose=True)
peneasy_sims.activate_pid('SPC')

In [ ]:
MATS = {'water' : ['water', 1, 1.00+00]}
SIZE = [10, 10, 10]
STEP = [0.4, 0.4, 0.4]  #cm
nhist = "1e6"

peneasy_inps.edit_voxSize(pid, SIZE, STEP)
peneasy_inps.edit_seed(pid, 12345, 54321)
peneasy_inps.edit_mat(pid, MATS, SIZE, STEP)
peneasy_inps.edit_source_nhist(pid, nhist)

In [ ]:
isotopes = ["N13", "O15", "F18", "Cu64", "Ga68", "Rb82", "I124"] #"C11", 

peneasy_inps.verbose = False
for iso in isotopes:
    print(f"Simulating {iso}")
    peneasy_inps.edit_isotope(pid, iso)
    peneasy_sims.simulate(pid, get_times=True, time_samples=1, output_dir="RESULTS/SPC-Study1-Water/PenEasy_xyz", final_file=f"{iso}")

# vGATE 9.x simulation

In [ ]:
pid = '9'
# Initialize GATE Input Editor
gate_inps = inputs.GATE(verbose=True)

# Initialize Simulators (identified by pid)
gate_sims = simuls.GATE(verbose=True)
gate_sims.activate_pid(pid, output_format='dat')

In [ ]:
MATS = {'water' : ['Water', 6, 1.00+00]}
SIZE = [10, 10, 10]
STEP = [0.4, 0.4, 0.4]  #cm
nhist = "1e6"

gate_inps.edit_voxSize(pid, SIZE, STEP)
gate_inps.edit_seed(pid, 12345, 54321)
gate_inps.edit_mat(pid, MATS, SIZE, STEP)
gate_inps.edit_source_nhist(pid, nhist)

In [ ]:
isotopes = ["C11", "N13", "O15", "F18", "Cu64", "Ga68", "Rb82", "I124"] #
res_folder = "RESULTS/SPC/Water/GATE93_xyz"

gate_inps.verbose = False
for iso in isotopes:
    print(f"Simulating {iso}")
    gate_inps.edit_isotope(pid, iso)
    gate_sims.simulate(pid, get_times=True, time_samples=1, output_dir=res_folder, final_file=f"{iso}")


In [ ]:
times_file = res_folder + "/" + pid + "_times.txt"
times = np.loadtxt(times_file).astype("str")
times[:,1], times[:,0] = times[:,0], isotopes
np.savetxt(times_file, times, fmt='%s')

# Results analysis

In [23]:
rthresh = {"C11" : 5, "N13" : 6, "O15" : 9, "F18" : 3, "Cu64" : 4, "Ga68" : 10, "Rb82" : 20, "I124" : 12}  #water

program_folder = "PHITS_xyz"
folder = f"RESULTS/SPC/Water/{program_folder}"
trim = int(1e6)

for iso in rthresh.keys():
    if program_folder == "GATE7_xyz" and iso not in ["C11", "O15", "F18"]:
        continue
    print(f"Filtering {iso}")
    threshold = rthresh[iso] #mm
    analysis.filter_rmax(f"{folder}/{iso}.dat.original", f"{folder}/{iso}.dat", threshold=threshold, trim=trim, fmt='%.6f')

Filtering C11
Filtered 'RESULTS/SPC/Water/PHITS_xyz/C11.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/C11.dat': 0 points removed
Filtering N13
Filtered 'RESULTS/SPC/Water/PHITS_xyz/N13.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/N13.dat': 1 points removed
Filtering O15
Filtered 'RESULTS/SPC/Water/PHITS_xyz/O15.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/O15.dat': 3 points removed
Filtering F18
Filtered 'RESULTS/SPC/Water/PHITS_xyz/F18.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/F18.dat': 0 points removed
Filtering Cu64
Filtered 'RESULTS/SPC/Water/PHITS_xyz/Cu64.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/Cu64.dat': 0 points removed
Filtering Ga68
Filtered 'RESULTS/SPC/Water/PHITS_xyz/Ga68.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/Ga68.dat': 9 points removed
Filtering Rb82
Filtered 'RESULTS/SPC/Water/PHITS_xyz/Rb82.dat.original' to 'RESULTS/SPC/Water/PHITS_xyz/Rb82.dat': 48 points removed
Filtering I124
Filtered 'RESULTS/SPC/Water/PHITS_xyz/I124.dat.original' to 'RESULTS/SPC

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

histo_bins = 201
histo_step = .01 #cm
isotopes = ["C11", "N13", "O15", "F18", "Cu64", "Ga68", "Rb82", "I124"] 

lowPRisotopes = ["C11", "N13", "Cu64", "F18"]
highPRisotopes = ["O15", "Ga68", "Rb82", "I124"]
lowPRlabels = {iso:[] for iso in lowPRisotopes}
highPRlabels = {iso:[] for iso in highPRisotopes}

results.active_results.clear()
for iso in isotopes:
    of = f"RESULTS/SPC-Study1-Water/GATE92_xyz/{iso}.dat"
    results.load(f"GATEv9.2 {iso}", of, [histo_bins]*3, [histo_step]*3)

    if iso in lowPRlabels:
        lowPRlabels[iso].append(f"GATEv9.2 {iso}")
    elif iso in highPRlabels:
        highPRlabels[iso].append(f"GATEv9.2 {iso}")

# for iso in isotopes:
#     of = f"RESULTS/SPC-Study1-Water/PHITS_xyz/{iso}.dat"
#     results.load(f"PHITS {iso}", of, [histo_bins]*3, [histo_step]*3)

#     of = f"RESULTS/SPC-Study1-Water/PenEasy_xyz/{iso}.dat"
#     results.load(f"PenEasy {iso}", of, [histo_bins]*3, [histo_step]*3)

#     if iso in lowPRlabels:
#         lowPRlabels[iso].append(f"PHITS {iso}")
#         lowPRlabels[iso].append(f"PenEasy {iso}")
#     elif iso in highPRlabels:
#         highPRlabels[iso].append(f"PHITS {iso}")
#         highPRlabels[iso].append(f"PenEasy {iso}")

# for iso in ["C11", "O15", "F18"]:
#     of = f"RESULTS/SPC-Study1-Water/GATE7_xyz/{iso}.dat"
#     results.load(f"GATE7 {iso}", of, [histo_bins]*3, [histo_step]*3)

#     if iso in lowPRlabels:
#         lowPRlabels[iso].append(f"GATE7 {iso}")
#     elif iso in highPRlabels:
#         highPRlabels[iso].append(f"GATE7 {iso}")

lowPRlabels_all = [label for iso in lowPRisotopes for label in lowPRlabels[iso]]
highPRlabels_all = [label for iso in highPRisotopes for label in highPRlabels[iso]]

In [ ]:
results.data_analysis()

In [ ]:
results.plot_aPSFx(sin=True, log_scale=True, labels=highPRlabels["O15"])

In [ ]:
results.plot_aPSFx(sin=True, log_scale=True, lim=6, labels=lowPRlabels_all, legend_size=8)
plt.show()
results.plot_aPSFx(sin=True, log_scale=True, lim=10, labels=highPRlabels_all, legend_size=8)
plt.show()

In [ ]:
results.plot_aPSF3D(sin=True, log_scale=True, labels=lowPRlabels_all, legend_size=8)
plt.show()
results.plot_aPSF3D(sin=True, log_scale=True, labels=highPRlabels_all, legend_size=8)
plt.show()


In [ ]:
results.plot_g3D(log_scale=False, labels=lowPRlabels_all)

In [ ]:
results.plot_G3D(labels=lowPRlabels_all)